In [1]:
from model import Model
from utils import fetch_dataset
from grakel.kernels import ShortestPath, VertexHistogram, EdgeHistogram

In [3]:
MUTAG = fetch_dataset("MUTAG", verbose=False)
EH = VertexHistogram(normalize=False)